In [1]:
import numpy as np
import pandas as pd

import env

In [2]:
properties_query = """
SELECT id, parcelid, bathroomcnt, bedroomcnt, calculatedfinishedsquarefeet, fips, unitcnt, taxvaluedollarcnt, taxamount, (taxamount/taxvaluedollarcnt) as tax_rate
FROM properties_2017
WHERE unitcnt = 1;
"""

In [3]:
url = env.get_db_url("zillow")

In [4]:
properties = pd.read_sql(properties_query, url)
properties

,id,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,unitcnt,taxvaluedollarcnt,taxamount,tax_rate
0,4,10879947,0.0,0.0,1776.0,6037.0,1.0,440101.0,5725.17,0.013009
1,10,11070347,4.0,4.0,3095.0,6037.0,1.0,192544.0,2460.72,0.012780
2,20,11324547,2.0,4.0,3633.0,6037.0,1.0,296425.0,6941.39,0.023417
3,21,11391347,0.0,0.0,4053.0,6037.0,1.0,511433.0,6840.34,0.013375
4,22,11395747,0.0,0.0,1442.0,6037.0,1.0,50689.0,1522.08,0.030028
...,...,...,...,...,...,...,...,...,...,...
1783868,2982265,13027311,4.0,4.0,4375.0,6037.0,1.0,422400.0,13877.56,0.032854
1783869,2982280,11366340,1.0,2.0,798.0,6037.0,1.0,469300.0,5764.45,0.012283
1783870,2982281,13010327,3.0,3.0,1526.0,6037.0,1.0,594022.0,7343.47,0.012362
1783871,2982282,12385768,4.0,4.0,2110.0,6037.0,1.0,554009.0,6761.20,0.012204


In [5]:
properties["id"].value_counts().max()

1

In [6]:
properties.parcelid.value_counts().max()

1

In [7]:
predictions_query = """
SELECT parcelid, MAX(transactiondate) as last_transaction_date
FROM predictions_2017
WHERE transactiondate >= "2017-05-01"
AND transactiondate <= "2017-06-30"
GROUP BY parcelid
ORDER BY last_transaction_date;
"""

In [8]:
predictions = pd.read_sql(predictions_query, url)
predictions

,parcelid,last_transaction_date
0,13909504,2017-05-01
1,17172225,2017-05-01
2,12252417,2017-05-01
3,12529922,2017-05-01
4,12319747,2017-05-01
...,...,...
21929,12321535,2017-06-30
21930,12070399,2017-06-30
21931,10975999,2017-06-30
21932,14647039,2017-06-30


In [9]:
predictions.parcelid.value_counts().max()

1

In [10]:
joined_query = """
SELECT prop.id, prop.parcelid, prop.bathroomcnt, prop.bedroomcnt, prop.calculatedfinishedsquarefeet, prop.fips, prop.unitcnt, prop.taxvaluedollarcnt, prop.taxamount, (prop.taxamount/prop.taxvaluedollarcnt) as taxrate, pred.transactiondate
FROM properties_2017 as prop
JOIN predictions_2017 as pred USING(parcelid)
WHERE prop.unitcnt = 1
AND pred.transactiondate >= "2017-05-01"
AND pred.transactiondate <= "2017-06-30"
ORDER BY pred.transactiondate;
"""

In [11]:
joined = pd.read_sql(joined_query, url)
joined

,id,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,unitcnt,taxvaluedollarcnt,taxamount,taxrate,transactiondate
0,733483,11056104,3.0,3.0,1572.0,6037.0,1.0,404060.0,4934.43,0.012212,2017-05-01
1,1629778,11503172,3.0,4.0,3041.0,6037.0,1.0,245311.0,2963.65,0.012081,2017-05-01
2,438499,11993956,2.0,3.0,1692.0,6037.0,1.0,649169.0,8063.11,0.012421,2017-05-01
3,1956514,11054736,2.0,4.0,1611.0,6037.0,1.0,89640.0,1144.73,0.012770,2017-05-01
4,838335,11206210,2.0,2.0,1078.0,6037.0,1.0,124006.0,2290.48,0.018471,2017-05-01
...,...,...,...,...,...,...,...,...,...,...,...
13304,1664929,12832732,3.0,3.0,1740.0,6037.0,1.0,436000.0,5192.45,0.011909,2017-06-30
13305,1948691,12945108,2.0,3.0,1536.0,6037.0,1.0,297097.0,3519.78,0.011847,2017-06-30
13306,1480299,11464823,3.0,4.0,2305.0,6037.0,1.0,579047.0,6996.21,0.012082,2017-06-30
13307,1092708,11534364,2.0,3.0,1997.0,6037.0,1.0,630933.0,7587.79,0.012026,2017-06-30


In [12]:
joined.parcelid.value_counts()

13067305    2
11175935    1
12583749    1
12958562    1
12641113    1
           ..
12901697    1
12506432    1
11251007    1
13073306    1
10717185    1
Name: parcelid, Length: 13308, dtype: int64

In [13]:
joined.fips.value_counts()

6037.0    13307
6059.0        2
Name: fips, dtype: int64

In [14]:
joined[joined.parcelid == 13067305]

,id,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,unitcnt,taxvaluedollarcnt,taxamount,taxrate,transactiondate
1234,159239,13067305,3.0,2.0,1149.0,6037.0,1.0,218619.0,3074.84,0.014065,2017-05-05
11354,159239,13067305,3.0,2.0,1149.0,6037.0,1.0,218619.0,3074.84,0.014065,2017-06-26


In [15]:
joined.drop(index=1234, inplace=True)

In [16]:
joined

,id,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,unitcnt,taxvaluedollarcnt,taxamount,taxrate,transactiondate
0,733483,11056104,3.0,3.0,1572.0,6037.0,1.0,404060.0,4934.43,0.012212,2017-05-01
1,1629778,11503172,3.0,4.0,3041.0,6037.0,1.0,245311.0,2963.65,0.012081,2017-05-01
2,438499,11993956,2.0,3.0,1692.0,6037.0,1.0,649169.0,8063.11,0.012421,2017-05-01
3,1956514,11054736,2.0,4.0,1611.0,6037.0,1.0,89640.0,1144.73,0.012770,2017-05-01
4,838335,11206210,2.0,2.0,1078.0,6037.0,1.0,124006.0,2290.48,0.018471,2017-05-01
...,...,...,...,...,...,...,...,...,...,...,...
13304,1664929,12832732,3.0,3.0,1740.0,6037.0,1.0,436000.0,5192.45,0.011909,2017-06-30
13305,1948691,12945108,2.0,3.0,1536.0,6037.0,1.0,297097.0,3519.78,0.011847,2017-06-30
13306,1480299,11464823,3.0,4.0,2305.0,6037.0,1.0,579047.0,6996.21,0.012082,2017-06-30
13307,1092708,11534364,2.0,3.0,1997.0,6037.0,1.0,630933.0,7587.79,0.012026,2017-06-30


In [17]:
joined[joined.parcelid == 13067305]

,id,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,unitcnt,taxvaluedollarcnt,taxamount,taxrate,transactiondate
11354,159239,13067305,3.0,2.0,1149.0,6037.0,1.0,218619.0,3074.84,0.014065,2017-06-26


In [18]:
joined.parcelid.value_counts().max()

1

In [19]:
california_fips = pd.read_csv("california_fips.csv")
california_fips.drop(columns="Unnamed: 0", inplace=True)
california_fips

,06000,California
0,6001,Alameda County
1,6003,Alpine County
2,6005,Amador County
3,6007,Butte County
4,6009,Calaveras County
5,6011,Colusa County
6,6013,Contra Costa County
7,6015,Del Norte County
8,6017,El Dorado County
9,6019,Fresno County


In [29]:
alternate_joined_query = """SELECT prop.id, prop.parcelid, prop.bathroomcnt, prop.bedroomcnt, prop.calculatedfinishedsquarefeet, prop.fips, plut.propertylandusedesc, prop.unitcnt, prop.taxvaluedollarcnt, prop.taxamount, (prop.taxamount/prop.taxvaluedollarcnt) as tax_rate, pred.transactiondate
FROM properties_2017 as prop
JOIN predictions_2017 as pred USING(parcelid)
JOIN propertylandusetype as plut USING(propertylandusetypeid)
WHERE plut.propertylandusedesc = "Single Family Residential"
AND pred.transactiondate >= "2017-05-01"
AND pred.transactiondate <= "2017-06-30"
ORDER BY pred.transactiondate;
"""

In [30]:
alt_joined = pd.read_sql(alternate_joined_query, url)
alt_joined

,id,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,propertylandusedesc,unitcnt,taxvaluedollarcnt,taxamount,tax_rate,transactiondate
0,1175489,14365030,2.5,3.0,1653.0,6059.0,Single Family Residential,NaN,605000.0,6185.28,0.010224,2017-05-01
1,733483,11056104,3.0,3.0,1572.0,6037.0,Single Family Residential,1.0,404060.0,4934.43,0.012212,2017-05-01
2,1629778,11503172,3.0,4.0,3041.0,6037.0,Single Family Residential,1.0,245311.0,2963.65,0.012081,2017-05-01
3,438499,11993956,2.0,3.0,1692.0,6037.0,Single Family Residential,1.0,649169.0,8063.11,0.012421,2017-05-01
4,1956514,11054736,2.0,4.0,1611.0,6037.0,Single Family Residential,1.0,89640.0,1144.73,0.012770,2017-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...
15031,1324679,14339902,3.0,5.0,2526.0,6059.0,Single Family Residential,NaN,458903.0,5718.72,0.012462,2017-06-30
15032,2963359,13940564,3.0,5.0,2735.0,6059.0,Single Family Residential,NaN,115387.0,1465.88,0.012704,2017-06-30
15033,1948691,12945108,2.0,3.0,1536.0,6037.0,Single Family Residential,1.0,297097.0,3519.78,0.011847,2017-06-30
15034,444575,14214719,3.0,5.0,2655.0,6059.0,Single Family Residential,NaN,746963.0,8065.50,0.010798,2017-06-30


In [31]:
alt_joined.fips.value_counts()

6037.0    9630
6059.0    4109
6111.0    1297
Name: fips, dtype: int64

In [32]:
alt_joined.unitcnt.value_counts()

1.0    9593
2.0      10
Name: unitcnt, dtype: int64

In [33]:
alt_joined.propertylandusedesc.value_counts()

Single Family Residential    15036
Name: propertylandusedesc, dtype: int64